In [ ]:
import cv2 as cv
import os
import matplotlib.pyplot as plt

In [ ]:

# =========================================
# 1️⃣ Dossiers
# =========================================
dataset_folder = "License Plates"
matricule_folder = "matricules"

# =========================================
# 2️⃣ Initialiser SIFT
# =========================================
sift = cv.SIFT_create()
bf = cv.BFMatcher()

# =========================================
# 3️⃣ Fonction extraction SIFT
# =========================================
def extract_sift(path):
    img = cv.imread(path)
    if img is None:
        return None, None, None
    
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    return img, kp, des

# =========================================
# 4️⃣ Construire base des voitures dataset
# =========================================
database = []

for file in os.listdir(dataset_folder):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        path = os.path.join(dataset_folder, file)
        img, kp, des = extract_sift(path)
        
        if des is not None:
            database.append({
                "filename": file,
                "image": img,
                "keypoints": kp,
                "descriptors": des
            })

print("Base dataset chargée :", len(database), "voitures")

# =========================================
# 5️⃣ Comparer avec dossier matricule
# =========================================
MATCH_THRESHOLD = 40  # à ajuster

for file in os.listdir(matricule_folder):
    
    if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue
    
    print("\n==============================")
    print("Vérification :", file)
    
    path = os.path.join(matricule_folder, file)
    query_img, kp_query, des_query = extract_sift(path)
    
    if des_query is None:
        continue
    
    best_score = 0
    best_match = None
    best_good = None
    
    # Comparer avec toutes les voitures dataset
    for item in database:
        
        matches = bf.knnMatch(des_query, item["descriptors"], k=2)
        
        good = []
        for pair in matches:
            if len(pair) == 2:
                m, n = pair
                if m.distance < 0.75 * n.distance:
                    good.append(m)
        
        score = len(good)
        
        if score > best_score:
            best_score = score
            best_match = item
            best_good = good
    
    # =========================================
    # Décision
    # =========================================
    if best_score >= MATCH_THRESHOLD:
        print("✅ AUTORISÉ")
        print("Correspond à :", best_match["filename"])
        print("Score :", best_score)
        
        result = cv.drawMatches(
            query_img, kp_query,
            best_match["image"], best_match["keypoints"],
            best_good, None, flags=2
        )
        
        plt.figure(figsize=(10,6))
        plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
        plt.title(f"{file} → {best_match['filename']} | Score={best_score}")
        plt.axis("off")
        plt.show()
        
    else:
        print("❌ REFUSÉ")
        print("Score max trouvé :", best_score)

Base dataset chargée : 35 voitures

Vérification : mat1.png
❌ REFUSÉ
Score max trouvé : 6

Vérification : mat2.png
❌ REFUSÉ
Score max trouvé : 5

Vérification : mat3.png
❌ REFUSÉ
Score max trouvé : 11

Vérification : mat4.png
❌ REFUSÉ
Score max trouvé : 7
